This is an auxiliary file. It generates the visuals that are referenced in other files in this folder.

In [187]:
from manim import *
import numpy as np

# Global variable to specify the number of inputs
NUM_INPUTS = 5  # You can change this to specify the number of inputs

class PerceptronScene(Scene):
    
    def label(self, label_separators):
        # Labels for each section with "Weighted Sum" changed to "Sum"
        sections = ["Inputs", "Weights", "Sum", "Bias", "Output"]
        
        # Ensure there's one more separator than the sections for the rightmost boundary
        if len(label_separators) != len(sections) + 1:
            raise ValueError("label_separators must have exactly one more element than sections.")
        
        # Calculate the middle points for label placement
        label_positions = [(label_separators[i] + label_separators[i + 1]) / 2 for i in range(len(sections))]
        
        # Create the vertical lines and labels but don't play the animations yet
        line_animations = []
        label_animations = []
        for i, x in enumerate(label_separators[1:-1]):  # Skip the first and last as they are edges, not separators
            line = DashedLine(start=3*UP, end=3*DOWN).move_to([x, 0, 0])
            line_animations.append(Create(line))
            # Labels are placed just below the top of the scene, adjusted y-coordinate slightly lower
            label = Text(sections[i], font_size=24).move_to([label_positions[i], 2.9, 0])
            label_animations.append(Write(label))
        
        # Play the animations for lines and labels all at once
        self.play(*line_animations, *label_animations)



    
    def draw_perceptron(self):
        # Where lines will be drawn, now just using X-axis values
        label_separators = []

        # Dynamic position setup based on the number of inputs
        input_positions = [3 * LEFT + i * UP for i in np.linspace(-(NUM_INPUTS - 1) / 2, (NUM_INPUTS - 1) / 2, NUM_INPUTS)]
        
        # Separator to the left of the inputs, storing only the X value
        label_separators.append(input_positions[0][0] - 1)

        perceptron_center = RIGHT  # Perceptron center position

        # Dynamic input labels creation based on the number of inputs, with 0-indexing
        input_labels = [MathTex(f"x_{i}", font_size=36).move_to(pos) for i, pos in enumerate(input_positions[::-1])]

        # Display the input labels simultaneously
        self.play(*[Write(label) for label in input_labels])

        # Calculate the distance for uniform arrow size
        arrow_length = np.linalg.norm(3 * RIGHT - perceptron_center) - 0.5  # Distance to edge of circle

        weights = []
        # Arrows from inputs to the perceptron, adjusted for the dynamic number of inputs
        for i, label in enumerate(input_labels):
            direction = perceptron_center - label.get_center()
            direction = direction / np.linalg.norm(direction)  # Normalize direction
            arrow_end = perceptron_center - direction * arrow_length * 0.5  # Stop at the circle's edge
            arrow_start = label.get_center() + direction * arrow_length * 0.25
            arrow = Arrow(start=arrow_start, end=arrow_end, stroke_width=3, buff=0, tip_length=0.2)
            weights.append(arrow)

        # Draw all weights simultaneously
        self.play(*[Create(arrow) for arrow in weights])

        # Separator between inputs and weights, using X values
        # Calculate the X position of the right edge of the subscript of the last input label
        right_edge_of_subscript = input_labels[-1].get_right()[0]
        weights_start_x = arrow_start[0]
        label_separators.append((right_edge_of_subscript + weights_start_x) / 2)

        # Perceptron circle creation
        perceptron_circle = Circle(radius=0.5).move_to(perceptron_center)
        self.play(Create(perceptron_circle))

        # Separator between weights and perceptron (weighted sum), using X values
        closest_arrow_tip = arrow_end[0]
        edge_of_perceptron = perceptron_center[0] - perceptron_circle.radius
        label_separators.append((closest_arrow_tip + edge_of_perceptron) / 2)
        
        # Sum symbol inside the perceptron
        sum_symbol = MathTex(r"\sum", font_size=18).move_to(perceptron_circle.get_center())
        self.play(Write(sum_symbol))

        # Arrow from the perceptron to the output
        output_arrow = Arrow(start=perceptron_circle.get_right() + RIGHT * arrow_length * 0.25, end=perceptron_circle.get_right() + RIGHT * arrow_length, stroke_width=3, buff=0, tip_length=0.2)
        self.play(Create(output_arrow))

        # Separator between perceptron and output, using X values
        edge_of_perceptron = perceptron_center[0] + perceptron_circle.radius
        output_start_x = output_arrow.get_start()[0]
        label_separators.append((edge_of_perceptron + output_start_x) / 2)

        # Output label
        output_label = MathTex("z", font_size=36).next_to(output_arrow.get_end(), RIGHT * arrow_length * 0.5)
        self.play(Write(output_label))

        # Separator between output arrow and output label, using X values
        tip_of_output_arrow = output_arrow.get_end()[0]
        beginning_of_output_label = output_label.get_left()[0]
        label_separators.append((tip_of_output_arrow + beginning_of_output_label) / 2)

        # Final separator, to the right of the output label
        final_separator = output_label.get_right()[0] + 1
        label_separators.append(final_separator)

        # Add labels and separators based on calculated positions
        self.label(label_separators)


    
    def construct(self):
        self.draw_perceptron()
        
        # Hold the final image
        self.wait(5)

In [188]:
# Play the visual
%manim -ql -v WARNING PerceptronScene